In [1]:
import math
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#sns.set_theme()

#generate layer space and layer radius of 2nd, 3rd and 4th layer based on dihedral angle 69 deg 
def layer_config(dihedral_angle):
    #We assume the 1st layer is a equilateral triangle
    #1. Compute edge angle respect to 1st layer
    edge_angle = np.arctan((math.sqrt(3)/6*np.tan(dihedral_angle))/(math.sqrt(3)/3))
    
    #On the edge, the nearest two beads closely contact with each other ==> edge length = 1
    #2. Compute layer space
    layer_num = 4
    layer_spacing = [0]
    for i in range(1, layer_num):
        layer_spacing.append(i*np.sin(edge_angle))
    
    #3. Compute layer radius. For simplicity, combine beads number and layer radius together. 
    layer_radius = []
    
    #beads_per_layer = [[21, 12, 3], [18], [15], [12, 3]]  #84 beads total, each layer: 36, 18, 15, 15, with fillings of the 1st and 4th layer
    beads_per_layer = [[24, 15, 6], [21], [18], [15, 6]]  #105 beads total, each layer: 45, 21, 18, 21, with fillings of the 1st and 4th layer
    #compute base layer first
    base_radius = []
    beads_in_base = [24, 15, 6] #beads with fillings of 1st layer
    for beads in beads_in_base:
        base_radius.append(np.sqrt(3) * int(beads / 3) / 3)
    #set up outer most radius reference for computing other layers' radius
    radius_ref = base_radius[0]
    radius_4th_outer = base_radius[1]
    radius_4th_inner = base_radius[2]
    layer_radius.append(list(zip(beads_in_base, base_radius)))
    
    layer_radius.append(list(zip([21], [radius_ref-1*np.cos(edge_angle)])))#2nd layer radius
    layer_radius.append(list(zip([18], [radius_ref-2*np.cos(edge_angle)]))) #3nd layer radius
    
    beads_in_top = [15, 6]
    layer_radius_top = [radius_ref-3*np.cos(edge_angle), radius_4th_inner * (radius_ref-3*np.cos(edge_angle))/radius_4th_outer] #4nd layer radius
    layer_radius.append(list(zip(beads_in_top, layer_radius_top)))
    
    return np.array(layer_spacing), layer_radius
    
    
    
    


def generate_capsid():
    coords = []
    trimer_vertices = []
    
    layer_spacing, layer_radius = layer_config(69*math.pi/180)

    # number of beads per layer
    #frame_per_layer = [[8, 5, 2], [7], [6], [5, 2]] #beads in one frame per layer

    for layer, layer_info in enumerate(layer_radius, start=0):
        z = layer_spacing[layer]
        
        for bead_info in layer_info:
            layer_beads = bead_info[0]
            layer_radius_each = bead_info[1]
            
            # Calculate the angles for equilateral triangles inscribed in a circle (0, 120, 240 deg)
            angles = np.linspace(0, 2*np.pi, 3, endpoint=False)
            
            # Calculate vertex coords for this layer, also add them to a global vertex list just in case
            layer_vertices = []
            for angle in angles:
                x = layer_radius_each * np.cos(angle)
                y = layer_radius_each * np.sin(angle)
                trimer_vertices.append((x, y, z))
                layer_vertices.append((x, y, z))
            

            for i in range(len(layer_vertices)):
                x1, y1, _ = layer_vertices[i]
                x2, y2, _ = layer_vertices[(i + 1) % len(layer_vertices)]  # Connect last vertex to first
                for j in range((layer_beads // 3)):  # Distribute beads along each side (doing it this way allows all coords to be in one array, but subtract 1 from the range arg if you want to plot the vertices seperately) 
                    #print("Placing bead {} of {} on face {} of layer {}" .format(j+1, (layer_beads // 3) - 1, i+1, layer))
                    # Interpolate coordinates between vertices
                    x = x1 + (x2 - x1) * (j) / (layer_beads // 3)
                    y = y1 + (y2 - y1) * (j) / (layer_beads // 3)
                    coords.append((x, y, z))

                                    
    return np.array(coords), np.array(trimer_vertices)



In [3]:
capsid_coords, vertices = generate_capsid()
print("Total bead number: " , len(capsid_coords))
print("Bottom layer edge length: ", np.linalg.norm(vertices[0] - vertices[1]))

Total bead number:  105
Bottom layer edge length:  7.999999999999998


In [5]:
#create input coordinate

column_size = 7
coords_template = np.zeros((len(capsid_coords), column_size))

#assign each column with values: x,y,z,type,charge,diameter,mass
for i in range(len(capsid_coords)):
    coords_template[i, 0]=capsid_coords[i, 0]
    coords_template[i, 1]=capsid_coords[i, 1]
    coords_template[i, 2]=capsid_coords[i, 2]  

coords_template[:,3]=int(1)
type_list = [45,66,47,50,69,52,72,54,57,75,59,78,61,64,81]       #attractive beads
for i in type_list:
    coords_template[i,3]=int(2)

coords_template[:,4]=int(0)
coords_template[:,5]=int(1)
coords_template[:,6]=int(1)

df = pd.DataFrame(coords_template, columns=['x', 'y', 'z', 'type', 'charge', 'diameter', 'mass'])
cols = ['type', 'charge', 'diameter', 'mass']
df[cols] = df[cols].applymap(np.int64)
print(df.to_string())
df.to_csv("test_trimer", sep='\t', index=True, index_label='index')

                x             y         z  type  charge  diameter  mass
0    4.618802e+00  0.000000e+00  0.000000     1       0         1     1
1    3.752777e+00  5.000000e-01  0.000000     1       0         1     1
2    2.886751e+00  1.000000e+00  0.000000     1       0         1     1
3    2.020726e+00  1.500000e+00  0.000000     1       0         1     1
4    1.154701e+00  2.000000e+00  0.000000     1       0         1     1
5    2.886751e-01  2.500000e+00  0.000000     1       0         1     1
6   -5.773503e-01  3.000000e+00  0.000000     1       0         1     1
7   -1.443376e+00  3.500000e+00  0.000000     1       0         1     1
8   -2.309401e+00  4.000000e+00  0.000000     1       0         1     1
9   -2.309401e+00  3.000000e+00  0.000000     1       0         1     1
10  -2.309401e+00  2.000000e+00  0.000000     1       0         1     1
11  -2.309401e+00  1.000000e+00  0.000000     1       0         1     1
12  -2.309401e+00  4.440892e-16  0.000000     1       0         

C:\Users\fanbsun\AppData\Local\Temp\1\ipykernel_11524\1812086956.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(np.int64)


In [7]:
#create input edge

edge_list = []
neighbor_list = [ [] for _ in range(len(capsid_coords))]
cnt = 0
nonbending_list1 = list(range(0, 108))
nonbending_list2 = [110,111,115,119,123,127,129,130,133,137,141,145,147,148,151,155,159,161,
                    171,172,173,175,176,190,191,193,194,195,197,211,212,214,215,216,218,232,
                    234,235,236,238,239,250,251,253,254,255,257,268,269,271,272,273,275,286,
                    288,289,290,293,296,299,302,303,305,308,311,314,315,317,320]
nonbending_list = nonbending_list1 + nonbending_list2


for i in range(len(capsid_coords)):
    for j in range(i+1, len(capsid_coords)):
        if np.linalg.norm(capsid_coords[i] - capsid_coords[j])<1.01:
            neighbor_list[i].append(j)
            neighbor_list[j].append(i)
            if cnt in nonbending_list:
                edge_list.append((i, j, 0, np.linalg.norm(capsid_coords[i] - capsid_coords[j])))
            else:
                edge_list.append((i, j, 1, np.linalg.norm(capsid_coords[i] - capsid_coords[j])))
            #edge_list.append((i, j, 0, np.linalg.norm(capsid_coords[i] - capsid_coords[j])))
            cnt = cnt+1

df = pd.DataFrame(edge_list, columns=['e1', 'e2', 'type', 'length'])
print(df.to_string())
df.to_csv("test_edge", sep='\t', index=True, index_label='index')

      e1   e2  type    length
0      0    1     0  1.000000
1      0   23     0  1.000000
2      0   45     0  1.000000
3      1    2     0  1.000000
4      1   23     0  1.000000
5      1   24     0  1.000000
6      1   45     0  0.972238
7      1   46     0  0.995897
8      2    3     0  1.000000
9      2   24     0  1.000000
10     2   25     0  1.000000
11     2   46     0  0.976065
12     2   47     0  0.991839
13     3    4     0  1.000000
14     3   25     0  1.000000
15     3   26     0  1.000000
16     3   47     0  0.979938
17     3   48     0  0.987826
18     4    5     0  1.000000
19     4   26     0  1.000000
20     4   27     0  1.000000
21     4   48     0  0.983859
22     4   49     0  0.983859
23     5    6     0  1.000000
24     5   27     0  1.000000
25     5   28     0  1.000000
26     5   49     0  0.987826
27     5   50     0  0.979938
28     6    7     0  1.000000
29     6   28     0  1.000000
30     6   29     0  1.000000
31     6   50     0  0.991839
32     6  

In [9]:
#create input face

triangle_list = []
cnt = 0
for i in range(len(capsid_coords)):
    for j in range(i, len(capsid_coords)):
        for k in range(j, len(capsid_coords)):
            if k in neighbor_list[i] and j in neighbor_list[i] and k in neighbor_list[j]:
                triangle_list.append((i, j, k, 0, 1))
                cnt = cnt+1

df = pd.DataFrame(triangle_list, columns=['t1', 't2', 't3', 'type','normal'])
print(df.to_string())
df.to_csv("test_triangle", sep='\t', index=True, index_label='index')

      t1   t2   t3  type  normal
0      0    1   23     0       1
1      0    1   45     0       1
2      0   23   45     0       1
3      1    2   24     0       1
4      1    2   46     0       1
5      1   23   24     0       1
6      1   23   45     0       1
7      1   24   46     0       1
8      1   45   46     0       1
9      2    3   25     0       1
10     2    3   47     0       1
11     2   24   25     0       1
12     2   24   46     0       1
13     2   25   47     0       1
14     2   46   47     0       1
15     3    4   26     0       1
16     3    4   48     0       1
17     3   25   26     0       1
18     3   25   47     0       1
19     3   26   48     0       1
20     3   47   48     0       1
21     4    5   27     0       1
22     4    5   49     0       1
23     4   26   27     0       1
24     4   26   48     0       1
25     4   27   49     0       1
26     4   48   49     0       1
27     5    6   28     0       1
28     5    6   50     0       1
29     5  